In [59]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [60]:
%autoreload 2

In [222]:
%matplotlib inline
from pathlib import Path
import functools
import itertools
from dataclasses import dataclass, fields

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams 

from matplottoy.data import fruit
#from matplottoy.encoders import color, position
from matplottoy.artists import point, bar, utils


spath = Path("../../../draft/figures/code/")
rcParams['figure.dpi'] = 200

In [223]:
data = {'fruit': ['apple', 'orange', 'lemon', 'lime'], 
       'calories':[95, 67, 17, 20], 
       'juice':[True, True, False, False]}
df = pd.DataFrame(data)

In [224]:
data = fruit.FruitSection(df)

In [225]:
df.index.inferred_type

'integer'

In [226]:
data.local_section({'calories':(17, 20)})

FruitSection(section=   fruit  calories  juice
2  lemon        17  False
3   lime        20  False)

In [227]:
data.projection('fruit').values()

array(['apple', 'orange', 'lemon', 'lime'], dtype=object)

In [269]:
# should K be up front or last?

V = {'height': bar.Mu(lambda x : x, 'calories'), 
     'position': bar.Mu(position.Nominal({'apple': 0, 'orange': 2, 'lemon': 4, 'lime': 6}), 'fruit'),
     'floor': bar.Mu(lambda x: np.repeat(0, len(x)), None), 
     'width': bar.Mu(lambda x: np.repeat(.8, len(x)), None),
     'facecolor': bar.Mu(color.Nominal({True: 'orange', False: 'blue'}), 'juice'), 
     'edgecolor':bar.Mu(lambda x: np.repeat('k', len(x)), None),
     'linewidth':bar.Mu(lambda x: np.repeat(2, len(x)), None),
     'linestyle':bar.Mu(lambda x: np.repeat((None, None), len(x)),  None)}

In [270]:
BB = bar.Bar()

In [271]:
BB.V

{'height': Mu(nu=None, F=None),
 'linewidth': Mu(nu=None, F=None),
 'floor': Mu(nu=None, F=None),
 'linestyle': Mu(nu=None, F=None),
 'edgecolor': Mu(nu=None, F=None),
 'width': Mu(nu=None, F=None),
 'facecolor': Mu(nu=None, F=None),
 'position': Mu(nu=None, F=None)}

In [272]:
BN = BB.compose_with_nu(**V)

In [274]:
BN.V

{'height': Mu(nu=<function <lambda> at 0x000002A5196B9160>, F='calories'),
 'linewidth': Mu(nu=<function <lambda> at 0x000002A5197C6940>, F=None),
 'floor': Mu(nu=<function <lambda> at 0x000002A5197C6C10>, F=None),
 'linestyle': Mu(nu=<function <lambda> at 0x000002A5197C6160>, F=None),
 'edgecolor': Mu(nu=<function <lambda> at 0x000002A5197C6EE0>, F=None),
 'width': Mu(nu=<function <lambda> at 0x000002A5197C6790>, F=None),
 'facecolor': Mu(nu=<matplottoy.encoders.color.Nominal object at 0x000002A5197D1A60>, F='juice'),
 'position': Mu(nu=<matplottoy.encoders.position.Nominal object at 0x000002A5197D1F70>, F='fruit')}

In [275]:
BB.V

{'height': Mu(nu=None, F=None),
 'linewidth': Mu(nu=None, F=None),
 'floor': Mu(nu=None, F=None),
 'linestyle': Mu(nu=None, F=None),
 'edgecolor': Mu(nu=None, F=None),
 'width': Mu(nu=None, F=None),
 'facecolor': Mu(nu=None, F=None),
 'position': Mu(nu=None, F=None)}

In [276]:
BN = BN.compose_with_nu(edgecolor=bar.Mu(F=None, nu=lambda _: 'darkgray'))

In [277]:
BB.V['edgecolor']

Mu(nu=None, F=None)

In [278]:
BN.V['edgecolor']

Mu(nu=<function <lambda> at 0x000002A5196A1DC0>, F=None)

In [279]:
BN.make_mu(data.local_section())

{'orientation': 'v',
 'height': array([95, 67, 17, 20], dtype=int64),
 'linewidth': array([2, 2, 2, 2]),
 'floor': array([0, 0, 0, 0]),
 'linestyle': array([None, None, None, None, None, None, None, None], dtype=object),
 'edgecolor': 'darkgray',
 'width': array([0.8, 0.8, 0.8, 0.8]),
 'facecolor': [RGBA(r=1.0, g=0.6470588235294118, b=0.0, a=1.0),
  RGBA(r=1.0, g=0.6470588235294118, b=0.0, a=1.0),
  RGBA(r=0.0, g=0.0, b=1.0, a=1.0),
  RGBA(r=0.0, g=0.0, b=1.0, a=1.0)],
 'position': array([0, 2, 4, 6])}

In [280]:
for (k, v) in BN.V.items():
    print(k, v.F, data.local_section().projection(v.F).values())
    v.nu(data.local_section().projection(v.F).values())

height calories [95 67 17 20]
linewidth None [nan nan nan nan]
floor None [nan nan nan nan]
linestyle None [nan nan nan nan]
edgecolor None [nan nan nan nan]
width None [nan nan nan nan]
facecolor juice [ True  True False False]
position fruit ['apple' 'orange' 'lemon' 'lime']


In [281]:
bar.QHatData(BN)

In [282]:
bwd = bar.QHatData(BN).compose_with_tau(data)

In [283]:
bwd.query()

FruitSection(section=    fruit  calories  juice
0   apple        95   True
1  orange        67   True
2   lemon        17  False
3    lime        20  False)

In [284]:
A = bar.GenericArtist(bwd)

In [285]:
dir(bwd.graphic)

['P',
 'V',
 '_V',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slotnames__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'compose_with_nu',
 'make_mu',
 'orientation',
 'qhat']

In [286]:
A.topArt

In [287]:
f = lambda x : np.zeros(len(x))
f(bwd.section.projection(None).values())

array([0., 0., 0., 0.])

In [291]:
A.draw(None)